In [ ]:
import asyncio
import logging
import sys

from lsst.ts import salobj

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
async def run_lifecycle_command(csc, command_str, settings_to_apply=None):
    try:
        cmd = getattr(csc, f"cmd_{command_str}")
        if command_str == "start" and settings_to_apply is not None:
            cmd.set(settingsToApply=settings_to_apply)
        ackcmd = await cmd.start(timeout=15)
        print(f"ack={ackcmd.ack}, ack.error={ackcmd.error}, ackcmd.result={ackcmd.result}")
    except salobj.AckError as ae:
        print(f"Failed with ack={ae.ackcmd.ack}, ack.error={ae.ackcmd.error}, ackcmd.result={ae.ackcmd.result}")

In [ ]:
d = salobj.Domain()
await asyncio.sleep(10)

In [ ]:
ATQueue = salobj.Remote(d, 'ScriptQueue', index=2)
MTQueue = salobj.Remote(d, 'ScriptQueue', index=1)
Watcher = salobj.Remote(d, 'Watcher')
ATScheduler = salobj.Remote(d, 'Scheduler', index=2)
MTScheduler = salobj.Remote(d, 'Scheduler', index=1)
await asyncio.gather(
    ATQueue.start_task,
    MTQueue.start_task,
    Watcher.start_task,
    ATScheduler.start_task,
    MTScheduler.start_task,
)

In [ ]:
#command = "start"
#command = "enable"
#command = "disable"
#command = "standby"
command = "exitControl"

In [ ]:
await run_lifecycle_command(Watcher, command, "ncsa")

In [ ]:
await run_lifecycle_command(ATQueue, command, "")

In [ ]:
await run_lifecycle_command(MTQueue, command, "")

In [ ]:
await run_lifecycle_command(ATScheduler, command, "")

In [ ]:
await run_lifecycle_command(MTScheduler, command, "")